In [5]:
import ee
print(ee.__version__)
ee.Authenticate() #4/1AX4XfWgp97_v97nZ28Mxbt6GIRB4toBqfXWBt4p--ypz1fDymld6_lu3V90
ee.Initialize()

0.1.285


Enter verification code:  4/1AX4XfWi8nmCZ_qTAK1aw632bjjndcRRL9SzZLrdRc1_1k8_zCiRRyhZAVWk



Successfully saved authorization token.


In [4]:
from IPython.display import Image

In [1]:
# 1. define functions: 
# reduce_SNIC_largesample takes a SNIC_cluster image (~10k clusters), vectorize them into polygons, 
# cut them by every 5000 vectors and append to a list.
# for each list, extract features for each polygons
# from train_image (spectral bands).  
def reduce_SNIC_largesample(snic_cluster_img, spectral_bands, vector_scale, selector, ee_reducer):
    
    def vec_index_list(i):
        index_seg = ee.FeatureCollection(vectors.toList(cut_length, i))
        return index_seg
    
    def extract(in_vec):
        cluster_signals = spectral_bands.reduceRegions(collection=ee.FeatureCollection(in_vec),
                                                      scale=10,
                                                      reducer=ee_reducer)  
        return ee.FeatureCollection(cluster_signals)
    
    
    spectral_bands = spectral_bands.select(selector)
    
    # raster to vector
    vectors = snic_cluster_img.reduceToVectors(crs=snic_cluster_img.projection(),
                                               scale=vector_scale,
                                               geometryType='polygon',
                                               eightConnected=False,
                                               labelProperty='clusters',
                                               maxPixels=1e11,
                                               reducer=None
                                              )
    
    # get rid of vector properties, so the following reduceRegions will return only spectral info
    vectors = vectors.select([])
    
    # cut vectors by 5000
    vec_size = vectors.size()
    cut_length = 5000
    
    n = ee.List.sequence(0, vec_size, cut_length)
    vec_list = n.map(vec_index_list)
    
    extracted = vec_list.map(extract)
    
    band_names = ee.FeatureCollection(extracted.get(0)).first().toDictionary().keys()
    
    all_ft = ee.FeatureCollection(extracted).flatten().set({'properties': {'band_order': band_names}})

    return all_ft
      

In [2]:
# SNIC_clustering()
def SNIC_clustering(train_image):
    seeds = ee.Algorithms.Image.Segmentation.seedGrid(30)
    snic = ee.Algorithms.Image.Segmentation.SNIC(image=train_image,
                                                #size=200
                                                 compactness=9,
                                                 connectivity=8,
                                                 neighborhoodSize=256,
                                                 seeds=seeds
                                                )
    snic_cluster = snic.select('clusters')
    return(snic_cluster)

In [24]:
# load earth engine assets

# load Zambia boudanry
zambia = ee.FeatureCollection('users/stxiong8/zambia_all_boundaries/Zam_national_boundary_epsg4326')

# make zambia a geometry
zambia = zambia.first().geometry()


grid_50x50 = ee.FeatureCollection('projects/AfricaLC_BULC/Sitian/Grid_System/grid_system_50x50')

In [3]:
# 2. define a year-training_dictionary 
train_dict = {
    2000: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2000',
    2001: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2001',
    2002: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2002',
    2003: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2003',
    2004: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2004',
    2005: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2005',
    2006: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2006',
    2007: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2007',
    2008: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2008',
    2009: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2009',
    2010: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2010',
    2011: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2011',
    2012: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2012',
    2013: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2013',
    2014: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2014',
    2015: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2015',
    2016: 'users/stxiong8/BULCU_Training_Image/ZMB_30m_Training_Image_2016'
  };

In [6]:
grid_50x50 = ee.FeatureCollection('projects/AfricaLC_BULC/Sitian/Grid_System/grid_system_50x50')
zambia = ee.FeatureCollection('users/stxiong8/zambia_all_boundaries/Zam_national_boundary_epsg4326')

In [20]:
# try out which grid element and its corresponding grid id
grid = ee.Feature(grid_50x50.toList(1,1386).get(0))
grid_id = grid.get('id').getInfo()
print(grid_id)

2261


Main workflow

In [21]:
for i in [1384,1386]:#range(10,23):#597
    
    if i%100==0:
        print(i)
    
    # get grid by index 0 to ~1500
    grid = ee.Feature(grid_50x50.toList(1, i).get(0)) # toList(1,i) means start from the ith, and fetch 1 feature 
    
    # get grid id (the first grid's id is 29 in my grid_50x50)
    grid_id = grid.get('id').getInfo()
    #print('grid_id: ', grid_id)
    
    # convert grid to geometry
    grid = grid.geometry()
    
    clust_list = []
    
    for year in range(2000, 2016):
        
        # get training_image, fill the holes (already treated, but in case) by 1, clip by grid, clip by zambia incase the grid falls outside zam
        train_image = ee.Image(train_dict[year]).unmask(1).clip(grid).clip(zambia)
        snic_cluster = SNIC_clustering(train_image)
        
        cluster_signals = reduce_SNIC_largesample(snic_cluster,
                                                train_image,
                                                30,
                                                train_image.bandNames(),# use all bands,can specify ['VV','VH','VV_d_VH','sar_inc']
                                                ee.Reducer.mean().combine(ee.Reducer.stdDev(),"",True))
        
        clusterer = ee.Clusterer.wekaKMeans(20).train(cluster_signals)
        # reduce ~10k clusters to 20
        clustered_cluster = cluster_signals.cluster(clusterer, 'new_cluster')
        clustered_cluster = clustered_cluster.select('new_cluster')
        
        # vector to image
        clust = clustered_cluster.reduceToImage(properties=['new_cluster'],
                                                reducer=ee.Reducer.first()).clip(grid).rename(['Year'+str(year)])
        
        clust_list.append(clust)
        
#         url = train_image.clip(grid).select(0).getThumbURL({
#             'min': 0, 'max': 20, 'region': grid, 'dimensions': 256,
#             'palette': ['blue', 'yellow', 'orange', 'red']})
#         print(url)
#         Image(url=url)
    
    clust_image_multiyear = ee.Image(clust_list[::-1])
    
#     task = ee.batch.Export.image.toDrive(image=clust_image_multiyear, 
#                                          description='grid_'+str(grid_id)+'_u-sd_reduced_SNIC'+'-20class',
#                                          scale=30,
#                                          region=grid,
#                                          folder='Reduced_SNIC_50x50_u-sd_20class',
#                                          fileNamePrefix='grid_'+str(grid_id)+'_u-sd_reduced_SNIC'+'-20class',
#                                          crs='EPSG:4326',
#                                          fileFormat='GeoTIFF')
    
    
    task = ee.batch.Export.image.toAsset(image=clust_image_multiyear, 
                                         description='grid_'+str(grid_id)+'_u-sd_reduced_SNIC'+'-20class', 
                                         assetId='users/stxiong8/Reduced_SNIC_2259n2261/grid_'+str(grid_id)+'_u-sd_reduced_SNIC'+'-20class', 
                                         region=grid, 
                                         scale=30, 
                                         crs='EPSG:4326', 
                                         maxPixels=1e13
                                        )
    
    
    task.start()


print('all tasks started')    
    

    

all tasks started


In [2]:
[year for year in range(2000, 2016)]

[2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015]